In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

In [2]:
all_rows = pd.DataFrame(columns=['phrase', 'is_ingredient'])

In [4]:
for x in tqdm(range(100,600)):
    page = requests.get("http://www.geniuskitchen.com/recipe/a-" + str(x))
    if(page.status_code == 404):
        continue
    soup = BeautifulSoup(page.content, 'html.parser')
    ingrs_str = [i.get_text() for i in soup.select('ul.ingredient-list')][0]
    ingrs = [s for s in ingrs_str.split('\n') if s]
    title = soup.select('h1')[0].get_text()
    directions = [d.get_text() for d in soup.select('.directions-inner li')]
    df_ingr = pd.DataFrame({'phrase':ingrs})
    df_ingr['is_ingredient'] = 1
    df_title = pd.DataFrame({'phrase':[title]})
    df_title['is_ingredient'] = 0
    df_directions = pd.DataFrame({'phrase':directions})
    df_directions['is_ingredient'] = 0
    all_rows = pd.concat([all_rows, df_ingr, df_title, df_directions], ignore_index=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [07:16<00:00,  1.14it/s]


In [8]:
all_rows.describe()


,is_ingredient
count,9344.000000
mean,0.463292
std,0.498677
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [9]:
all_rows['phrase'] = all_rows['phrase'].str.lower()
all_rows.head()

,phrase,is_ingredient
0,1 lb stale bread (loaf),1.0
1,1 1⁄3 cups milk,1.0
2,3 eggs,1.0
3,4 ounces evaporated milk,1.0
4,1⁄4 cup butter,1.0


In [11]:
all_rows.to_csv('recipe_phrases.csv')
